In [291]:
import os


In [292]:
%pwd

'C:\\Users\\shwet\\Documents\\CV and applications\\Projects\\ML Project with MLFlow and AWS EC2\\ML-Project-with-MLFlow-and-AWS-EC2'

In [293]:
os.chdir("../")

In [294]:
%pwd

'C:\\Users\\shwet\\Documents\\CV and applications\\Projects\\ML Project with MLFlow and AWS EC2'

In [295]:
#os.chdir("c:\\Users\\shwet\\Documents\\CV and applications\\Projects\\ML Project with MLFlow and AWS EC2")
os.chdir("C:\\Users\\shwet\\Documents\\CV and applications\\Projects\\ML Project with MLFlow and AWS EC2\\ML-Project-with-MLFlow-and-AWS-EC2")

In [296]:
%pwd

'C:\\Users\\shwet\\Documents\\CV and applications\\Projects\\ML Project with MLFlow and AWS EC2\\ML-Project-with-MLFlow-and-AWS-EC2'

In [297]:
ls

 Volume in drive C is OS
 Volume Serial Number is 9C0D-DA84

 Directory of C:\Users\shwet\Documents\CV and applications\Projects\ML Project with MLFlow and AWS EC2\ML-Project-with-MLFlow-and-AWS-EC2

15-01-2025  15:34    <DIR>          .
15-01-2025  15:41    <DIR>          ..
14-01-2025  22:08    <DIR>          .github
14-01-2025  21:42             3,586 .gitignore
14-01-2025  22:11                 0 app.py
15-01-2025  15:34    <DIR>          artifacts
14-01-2025  22:08    <DIR>          config
14-01-2025  22:11                 0 Dockerfile
15-01-2025  13:22    <DIR>          logs
14-01-2025  22:11                 0 main.py
15-01-2025  15:34                 8 params.yaml
15-01-2025  12:56               314 README.md
14-01-2025  22:24               196 requirements.txt
15-01-2025  12:56    <DIR>          research
15-01-2025  15:34                20 schema.yaml
14-01-2025  22:24               935 setup.py
14-01-2025  22:24    <DIR>          src
14-01-2025  22:11             1,617 templat

### Update Entity (Return Type of variables mentioned in config.yaml)

In [298]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

### To read config, params, schema files

In [299]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml,create_directories
from mlProject.config import *



### Update Configuration manager

In [300]:
from pathlib import Path

CONFIG_FILE_PATH = Path("C:\\Users\\shwet\\Documents\\CV and applications\\Projects\\ML Project with MLFlow and AWS EC2\\ML-Project-with-MLFlow-and-AWS-EC2\\config\\config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")
SCHEMA_FILE_PATH = Path("schema.yaml")

In [301]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

## Update Data Ingestion Component

In [302]:
import mlProject.utils.common
print(dir(mlProject.utils.common))


['Any', 'BoxValueError', 'ConfigBox', 'Path', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'create_directories', 'ensure_annotations', 'get_size', 'joblib', 'json', 'load_bin', 'load_json', 'logger', 'os', 'read_yaml', 'save_bin', 'save_json', 'yaml']


In [303]:
from importlib import reload
import mlProject.utils.common
reload(mlProject.utils.common)


<module 'mlProject.utils.common' from 'c:\\users\\shwet\\documents\\cv and applications\\projects\\ml project with mlflow and aws ec2\\ml-project-with-mlflow-and-aws-ec2\\src\\mlProject\\utils\\common.py'>

In [304]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size



In [305]:
# it will create Data ingestion directory and download data
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  
    

## Create Pipeline

In [306]:
try:
    config= ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e
    

[2025-01-15 16:00:09,125: INFO: common: yaml file: C:\Users\shwet\Documents\CV and applications\Projects\ML Project with MLFlow and AWS EC2\ML-Project-with-MLFlow-and-AWS-EC2\config\config.yaml loaded successfully]
[2025-01-15 16:00:09,130: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-15 16:00:09,133: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-15 16:00:09,136: INFO: common: created directory at: artifacts]
[2025-01-15 16:00:09,139: INFO: common: created directory at: artifacts/data_ingestion]
[2025-01-15 16:00:09,142: INFO: 112280761: File already exists of size: ~ 25 KB]
